In [ ]:
import os 
import json 
import pandas as pd
import traceback

In [ ]:
!pip install langchain_groq

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
KEY =os.getenv("GROQ_API_KEY")

In [ ]:
# from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key = KEY,
    model="mixtral-8x7b-32768",
    temperature=0.5
)